# Exploring dynamic difficulty adjustment (adaptive habit target logic)

Main goals with the system:
- Setting targets high enough for the user to improve, and low enough to consistently succeed.
- Prioritizing consistency over growth, and treating growth as a biproduct of consistent success.
- Drop the target quickly when user "crash" to spark motivation and minimize the friction.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import csv
from dataclasses import dataclass
from typing import List, Optional, Tuple
from datetime import date, datetime

: 

### Data classes

In [ ]:
@dataclass
class DataPoint:
    log_date: Optional[date] = None
    # difficulty: Optional[int] = None    # User-inserted value of perceived difficulty (easy, moderate, or hard).
    value: Optional[float] = None
    target: Optional[float] = None

@dataclass
class History:
    data_points: List[DataPoint]
    alpha: float

MIN_TARGET=2.0

### Load dataset

In [ ]:
df = pd.read_csv("data/06_dataset.csv")
df['log_date'] = pd.to_datetime(df['log_date'])
df.columns = df.columns.str.strip()

### Plotting dataset

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(df['log_date'], df['target'], label='Target Value', color='orange', linestyle='--', marker='o')
plt.plot(df['log_date'], df['value'], label='Value (Actual)', color='blue', marker='x')

plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()

## Holt's Linear Trend 
*aka Double Exponential Smoothing*, extends simple exponential smoothing of time series data with an underlying trend. Unlike moving average Holt's Linear Trend capture the rate of change, or trend, in the data. Used to forecast data.

**Definition:**
- $L_t$ (Level)
- $T_t$ (Trend)
- $\alpha \in [0, 1]$ (Smoothing for level)
- $\beta \in [0, 1]$ (Smoothing for trend)
- $Y_t$: (Actual Value)

$$\textbf{Level Equation:}$$
$$L_t = \alpha Y_t + (1 - \alpha)(L_{t-1} + T_{t-1})$$

$$\textbf{Trend Equation:}$$
$$T_t = \beta (L_t - L_{t-1}) + (1 - \beta)T_{t-1}$$

$$\textbf{Forecast Equation:}$$
$$\hat{Y}_{t+h} = L_t + (h \times T_t)$$

In [ ]:
def hlt(
    old_l: float, 
    old_t: float, 
    a: float,
    b: float,
    y: float,
):
    h = 1
    l = a*y + (1 - a)*(old_l + old_t)
    t = b*(l - old_l) + (1 - b)*old_t
    new_y = l + (h*t)
    return l, t, new_y

In [ ]:
ALPHA: float = 0.2
BETA: float = 0.4
level = df['value'].iloc[0]
trend = df['value'].iloc[1] - df['value'].iloc[0]

forecasts = [None]
first_forecast = level + trend
forecasts.append(first_forecast)

for i in range(1, len(df)):
    y = df['value'].iloc[i]
    level, trend, next_y = hlt(level, trend, ALPHA, BETA, y)
    forecasts.append(next_y)

last_date = df['log_date'].iloc[-1]
new_row = pd.DataFrame({'log_date': [last_date + pd.Timedelta(days=1)]})
df = pd.concat([df, new_row], ignore_index=True)

df['holt'] = forecasts
#df['diff'] = df['value'] - df['target']

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(df['log_date'], df['target'], label='Target Value', color='orange', linestyle='--', marker='o')
plt.plot(df['log_date'], df['value'], label='Value (Actual)', color='blue', marker='x')
plt.plot(df['log_date'], df['holt'], label="Holt's Linear Trend Forecast", color='red', marker='.')

plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()

## "Rubber Banding"
*A mechanic from the game Mario Kart that regulate the difficulty to keep the player engaged.* It's a form of Dynamic Difficulty Adjustment (DDA), and the goal is to keep the racers (racecar players) close together, like they are connected by a rubber band. Far ahead? ...speeds the others up, far behind? ...slows the others down, to let you catch up.

Inputs:
- $R$ (Rank)
- $\Delta d$: Distance difference
- $k \in [0,1]$: (Elasticity Coefficient)

Bot Velocity Equation:

Let $v_t$ be the new velocity and $v_0$ be the default velocity.
$$\text{Rank Scalar: } \lambda = R \cdot k \quad$$
$$v_t = v_0 \times (1 + \lambda \cdot \Delta d)$$

### Applied to habit targets
If we apply this method to the target adaptation, we compute next target instead of racing bot velocity.

**Definition:**
- $H_t$ (Holt's Linear Trend Forecast): Acts as the user's predicted *'true capability'*.
- $Y_t$ (Actual Value)
- $V_t$ (Target)
- $k$ (Elasticity Coefficient)

$$\text{Next Target Equation:}$$
$$V_t = H_t \times \left(1 + k \cdot \frac{Y_{t-1} - V_{t-1}}{V_{t-1}} \right)$$

The "rank" could later be the User-inserted value of perceived difficulty (easy, moderate, or hard). Or "rank" could be a measurement of 'Last week's performance compared to last 3 months'. The third alternative is to let "rank" be the 'current measured consistency'. "Rank" would then modulate $k$.

In [ ]:
def target_rubber_band(
    h: float,
    old_value: float,
    old_target: float,
    k: float
):
    completion: float = old_value - old_target
    completion_percentage: float = completion / old_target
    new_target = h * (1 + k*completion_percentage)
    return new_target

In [ ]:
ELASTICITY: float = 0.8
ALPHA: float = 0.2
BETA: float = 0.4
level = df['value'].iloc[0]
trend = df['value'].iloc[1] - df['value'].iloc[0]

rb_targets = [None]
forecasts = [None]
first_forecast = level + trend
forecasts.append(first_forecast)
rb_targets.append(None)

for i in range(1, len(df)):
    value = df['value'].iloc[i]
    level, trend, next_holt = hlt(level, trend, ALPHA, BETA, value)
    forecasts.append(next_holt)

    target = df['target'].iloc[i]
    next_target = target_rubber_band(next_holt, value, target, ELASTICITY)
    rb_targets.append(next_target)

last_date = df['log_date'].iloc[-1]
new_row = pd.DataFrame({'log_date': [last_date + pd.Timedelta(days=1)]})
df = pd.concat([df, new_row], ignore_index=True)

df['holt'] = forecasts
df['rb_target'] = rb_targets
#df['diff'] = df['value'] - df['target']

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(df['log_date'], df['target'], label='Target Value', color='orange', linestyle='--', marker='o')
plt.plot(df['log_date'], df['value'], label='Value (Actual)', color='blue', marker='x')
# plt.plot(df['log_date'], df['holt'], label="Holt's Linear Trend Forecast", color='red', marker='.')
plt.plot(df['log_date'], df['rb_target'], label="Next Target", color='purple', marker='.')

plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()

### Asymmetric Elasticity
As seen in the plot above, the calculation of the next target, using the rubber band method, were a bit too sensitive to high "spikes" and not enough elastic when user performs under the target.

To solve this we introduce two different Elasticity Coefficients for values above target and values below target.

For example:
- $k_{below} = 0.9$ (Case: user is failing. Result: drop target fast)
- $k_{above} = 0.1$ (Case: user is succeeding. Result: dampen high spikes / less elastic)

In [ ]:
def target_rubber_band_v2(
    h: float,
    old_value: float,
    old_target: float,
    k_above: float,
    k_below: float
):
    completion: float = old_value - old_target
    completion_percentage: float = completion / old_target

    k = 0.5
    if completion_percentage < 0:
        k = k_below
    else:
        k = k_above

    new_target = h * (1 + k*completion_percentage)
    return max(MIN_TARGET, new_target)

In [ ]:
ELASTICITY_BELOW: float = 0.9
ELASTICITY_ABOVE: float = 0.1
ALPHA: float = 0.2
BETA: float = 0.4
level = df['value'].iloc[0]
trend = df['value'].iloc[1] - df['value'].iloc[0]
target = df['target'].iloc[0]

rb_targets = [None]
forecasts = [None]
first_forecast = level + trend
forecasts.append(first_forecast)
rb_targets.append(None)

for i in range(1, len(df)):
    value = df['value'].iloc[i]
    # target = df['target'].iloc[i]
    
    level, trend, next_holt = hlt(level, trend, ALPHA, BETA, value)
    forecasts.append(next_holt)

    next_target = target_rubber_band_v2(
        next_holt, value, target, ELASTICITY_ABOVE, ELASTICITY_BELOW
    )
    rb_targets.append(next_target)
    target = next_target

last_date = df['log_date'].iloc[-1]
new_row = pd.DataFrame({'log_date': [last_date + pd.Timedelta(days=1)]})
df = pd.concat([df, new_row], ignore_index=True)

df['holt'] = forecasts
df['rb_target'] = rb_targets
#df['diff'] = df['value'] - df['target']

In [ ]:
rb_df = df.iloc[:91]

plt.figure(figsize=(15, 6))

plt.plot(df['log_date'], df['target'], label='Target Value', color='orange', linestyle='--', marker='o')
plt.plot(df['log_date'], df['value'], label='Value (Actual)', color='blue', marker='x')
# plt.plot(df['log_date'], df['holt'], label="Holt's Linear Trend Forecast", color='red', marker='.')
plt.plot(rb_df['log_date'], rb_df['rb_target'], label="Next Target", color='purple', marker='.')

plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()

## Stabilizing & Damping
From above results we can see that the calculated 'next targets' can get out of control if the user's activity is unpredictable or includes temporary 'spikes'. 

To make the target adaptation system more reliable we can introduce some rules, such as damping and clamping.

1. Problem: If the user, for one day, logs a value very high above the target, the system now puts the next target way too high for the user to be consistently succeed at.

2. Problem: If the user has big differences in logged value from day to day, the new target can move with a massive spread and looks very unstable, which can be confusing to the user.

In [ ]:
def target_rubber_band_v3(
    h: float,
    value: float,
    old_target: float,
    k_above: float,
    k_below: float,
    max_increase: float,
    max_decrease: float
):  
    completion: float = value - old_target
    completion_percentage: float = completion / old_target
    k = k_above if completion_percentage >= 0 else k_below

    new_target = h * (1 + k*completion_percentage)

    upper_boundary = old_target * (1 + max_increase)
    lower_boundary = old_target * (1 - max_decrease)

    target = max(lower_boundary, min(new_target, upper_boundary))
    
    return max(max(MIN_TARGET, 0.5*h), target)

In [ ]:
ELASTICITY_BELOW: float = 0.9
ELASTICITY_ABOVE: float = 0.1
ALPHA: float = 0.3
BETA: float = 0.1
VALUE_INCREASE_CAP: float = 1.5
MAX_INCREASE: float = 1.1
MAX_DECREASE: float = 0.9

level = df['value'].iloc[0]
trend = df['value'].iloc[1] - df['value'].iloc[0]
target = df['target'].iloc[0]

rb_targets = [None]
forecasts = [None]
first_forecast = level + trend
forecasts.append(first_forecast)
rb_targets.append(target)

for i in range(1, len(df)):
    value = df['value'].iloc[i]
    # target = df['target'].iloc[i]

    # Holt's
    level, trend, next_holt = hlt(level, trend, ALPHA, BETA, value)
    forecasts.append(next_holt)

    # Kill momentum on failure
    if value < target * 0.95:
        trend = 0.0
        next_holt = level

    # Value input limiter
    cap = target * VALUE_INCREASE_CAP
    if value > cap:
        value = cap

    # Rubber Band
    next_target = target_rubber_band_v3(
        next_holt, value, target, ELASTICITY_ABOVE, 
        ELASTICITY_BELOW, MAX_INCREASE, MAX_DECREASE
    )
    
    target = next_target
    rb_targets.append(target)


last_date = df['log_date'].iloc[-1]
new_row = pd.DataFrame({'log_date': [last_date + pd.Timedelta(days=1)]})
df = pd.concat([df, new_row], ignore_index=True)

df['holt'] = forecasts
df['rb_target'] = rb_targets
#df['diff'] = df['value'] - df['target']

In [ ]:
rb_df = df.iloc[:91]

plt.figure(figsize=(15, 6))

plt.plot(df['log_date'], df['target'], label='Target Value', color='orange', linestyle='--', marker=' ')
plt.plot(df['log_date'], df['value'], label='Value (Actual)', color='blue', marker=' ')
plt.plot(df['log_date'], df['holt'], label="Holt's Linear Trend Forecast", color='red', marker=' ')
plt.plot(rb_df['log_date'], rb_df['rb_target'], label="Next Target", color='purple', marker=' ')

plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()

## Elasticity Coefficient Progression
Adding a sigmoid function to growth elasticity to ensure long-term users aren't punished with endless difficult growth, while keeping the sensitivity high early. Growth Elasticity will drop when users consistently reach target (keeping a streak).

In [ ]:
# Calculates Elasticity Coefficient (k) -above based on streak
def elasticity_above_sigmoid(
    streak: int, 
    k_max: float = 0.4,
    k_min: float = 0.05,
    inflection_point_days: int = 14
):
    steepness = 0.3
    sigmoid_factor = 1 / (1 + np.exp(-steepness * (streak - inflection_point_days)))
    return k_max - (k_max - k_min) * sigmoid_factor


def elasticity_below_linear(
    streak: int, 
    k_max: float = 0.9, 
    k_min: float = 0.2, 
    protect_threshold_days: int = 21
):
    if streak <= 1:
        return k_max
    if streak >= protect_threshold_days:
        return k_min
    ratio = streak / protect_threshold_days
    return k_max - (ratio * (k_max - k_min))

### Adding streak counting in main loop

In [ ]:
ELASTICITY_BELOW: float = 0.9
ELASTICITY_ABOVE: float = 0.1
ALPHA: float = 0.1
BETA: float = 0.1
VALUE_INCREASE_CAP: float = 1.5
MAX_INCREASE: float = 1.1
MAX_DECREASE: float = 0.9

level = df['value'].iloc[0]
trend = df['value'].iloc[1] - df['value'].iloc[0]
target = df['target'].iloc[0]

rb_targets = [None]
forecasts = [None]
first_forecast = level + trend
forecasts.append(first_forecast)
rb_targets.append(target)

current_streak = 0

for i in range(1, len(df)):
    value = df['value'].iloc[i]
    # target = df['target'].iloc[i]
        
    # Holt's
    level, trend, next_holt = hlt(level, trend, ALPHA, BETA, value)
    forecasts.append(next_holt)

    # Kill momentum on failure
    if value < target * 0.95:
        trend = trend * 0.5
        next_holt = level

    is_success = value >= target
    if is_success:
        k = elasticity_above_sigmoid(current_streak)
        current_streak += 1
    else:
        k = elasticity_below_linear(current_streak)
        current_streak = 0

    # Value input limiter
    cap = target * VALUE_INCREASE_CAP
    if value > cap:
        value = cap

    # Rubber Band
    next_target = target_rubber_band_v3(
        next_holt, value, target, 
        k if is_success else None, 
        k if not is_success else None, 
        MAX_INCREASE,
        MAX_DECREASE
    )
    
    target = next_target
    rb_targets.append(next_target)


last_date = df['log_date'].iloc[-1]
new_row = pd.DataFrame({'log_date': [last_date + pd.Timedelta(days=1)]})
df = pd.concat([df, new_row], ignore_index=True)

df['holt'] = forecasts
df['rb_target'] = rb_targets
#df['diff'] = df['value'] - df['target']

In [ ]:
rb_df = df.iloc[:91]

plt.figure(figsize=(20, 10))

#plt.plot(df['log_date'], df['target'], label='Target Value', color='orange', linestyle='--', marker='o')
plt.plot(df['log_date'], df['value'], label='Value (Actual)', color='blue', marker='x')
plt.plot(df['log_date'], df['holt'], label="Holt's Linear Trend Forecast", color='red', marker='.')
plt.plot(rb_df['log_date'], rb_df['rb_target'], label="Next Target", color='purple', marker='.')

plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()